In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
directory_path = "/home/kwat/garden/data/cancer_cell_line/ctrp/"

kraft.path.path(directory_path)

In [ ]:
compound_table = pd.read_csv(
    "{}/v2/v20.meta.per_compound.txt".format(directory_path),
    sep="\t",
    usecols=(0, 1, 6, 7),
).to_numpy()

In [ ]:
cell_line_table = pd.read_csv(
    "{}v2/v20.meta.per_cell_line.txt".format(directory_path), sep="\t", usecols=(0, 1)
).to_numpy()

## Make _x\_compound

In [ ]:
compounds = compound_table[:, 1]

assert not kraft.array.check_has_duplicate(compounds)

In [ ]:
gene_x_compound = kraft.series.binarize(
    pd.Series(compound_table[:, 2], index=compounds)
)

gene_x_compound.index.name = "Gene"

gene_x_compound.to_csv("{}/gene_x_compound.tsv".format(directory_path), sep="\t")

gene_x_compound

In [ ]:
activity_x_compound = kraft.series.binarize(
    pd.Series(compound_table[:, 3], index=compounds)
)

activity_x_compound.index.name = "Activity"

activity_x_compound.to_csv(
    "{}/activity_x_compound.tsv".format(directory_path), sep="\t"
)

activity_x_compound

## Make compound_x_cell_line

In [ ]:
number_axis_0_axis_1 = pd.read_csv(
    "{}new-abs-auc-with-qc.txt".format(directory_path), sep="\t", usecols=(0, 3, 5)
).to_numpy()

compound_x_cell_line = kraft.dataframe.pivot(
    number_axis_0_axis_1[:, 1].astype(int),
    number_axis_0_axis_1[:, 2].astype(int),
    number_axis_0_axis_1[:, 0],
    function=min,
)

compound_id_to_name = {id_: name for id_, name in compound_table[:, :2]}

cell_line_id_to_name = {id_: name for id_, name in cell_line_table}

compound_x_cell_line.index = (
    compound_id_to_name[id_] for id_ in compound_x_cell_line.index.to_numpy()
)

compound_x_cell_line.columns = (
    cell_line_id_to_name[id_] for id_ in compound_x_cell_line.columns.to_numpy()
)

compound_x_cell_line.index.name = "Compound"

compound_x_cell_line.columns.name = "Cell Line"

kraft.dataframe.error_axes(compound_x_cell_line)

compound_x_cell_line.to_csv(
    "{}/compound_x_cell_line.tsv".format(directory_path), sep="\t"
)

compound_x_cell_line